In [1]:
import sys
import os

# Add the parent directory of the current working directory to sys.path
sys.path.append(os.path.abspath(".."))

In [9]:
print(os.getcwd())

/home/ec2-user/SageMaker/Fine_Tune_LLMs/Big/src


In [21]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 170.9 MB/s eta 0:00:0000:0100:01


In [2]:
import yaml
import boto3
import src
from src.document_processor.loader import DocumentLoader
from src.document_processor.chunker import DocumentChunker
from src.document_processor.cleaner import TextCleaner
from src.embeddings.embedding_manager import EmbeddingManager
from src.question_generation.generator import EnhancedQuestionGenerator
import logging


with open('/home/ec2-user/SageMaker/Fine_Tune_LLMs/Big/config/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Setup logging
logging.basicConfig(
    level=getattr(logging, config['logging']['level']),
    format=config['logging']['format']
)

logger = logging.getLogger(__name__)

# Initialize AWS client
bedrock_client = boto3.client('bedrock-runtime')

# Initialize components
loader = DocumentLoader()
chunker = DocumentChunker(
    chunk_size=config['document_processing']['chunk_size'],
    chunk_overlap=config['document_processing']['chunk_overlap']
)
cleaner = TextCleaner()

embedding_manager = EmbeddingManager(
    bedrock_client,
    model_id=config['embedding']['model_id']
)

generator = EnhancedQuestionGenerator(
    llm_client=bedrock_client,
    model_id=config['question_generation']['model_id'],
    embedding_manager=embedding_manager,
    max_tokens=config['question_generation']['max_tokens'],
    temperature=config['question_generation']['temperature']
)

2024-11-08 11:02:15,337 - botocore.credentials - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
/home/ec2-user/SageMaker/Fine_Tune_LLMs/Big/src/embeddings/embedding_manager.py:12: LangChainDeprecationWarning: The class `BedrockEmbeddings` was deprecated in LangChain 0.2.11 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import BedrockEmbeddings``.
  self.embeddings = BedrockEmbeddings(


In [10]:
# Process documents
documents = loader.load_document("/home/ec2-user/SageMaker/Fine_Tune_LLMs/Big/Physics_NuclMed.pdf")
logger.info("Documents loaded")

# Clean and chunk documents
cleaned_documents = []
for doc in documents:
    doc.page_content = cleaner.clean_text(doc.page_content)
    cleaned_documents.append(doc)
logger.info("Documents cleaned")

chunks = chunker.chunk_documents(cleaned_documents)
logger.info(f"Documents chunked into {len(chunks)} chunks")

# Create embeddings
embedding_manager.create_embeddings(chunks)
logger.info("Embeddings created")

2024-11-08 11:13:01,778 - __main__ - INFO - Documents loaded
2024-11-08 11:13:01,927 - __main__ - INFO - Documents cleaned
2024-11-08 11:13:02,374 - __main__ - INFO - Documents chunked into 2167 chunks
2024-11-08 11:15:30,794 - src.embeddings.embedding_manager - INFO - Embeddings created and stored in FAISS
2024-11-08 11:15:30,795 - __main__ - INFO - Embeddings created


In [11]:
import pandas as pd
# Generate questions
questions = generator.generate_questions_from_docs(
    chunks,
    num_questions=100
)
logger.info(f"{len(questions)} question-answer pairs generated")

# Prepare data for DataFrame
qa_data = [{"Question": qa.question, "Answer": qa.answer} for qa in questions]

# Create DataFrame
qa_df = pd.DataFrame(qa_data)

2024-11-08 11:21:15,684 - __main__ - INFO - 100 question-answer pairs generated


,Question,Answer
0,What is the process for a first-time user to a...,"For a first-time user, the process to access t..."
1,What are the contact details provided for tech...,The contact details provided for technical ass...
2,What is the role of physics in nuclear medicine?,"According to the table of contents, the role o..."
3,What topics are covered in Chapter 2?,Chapter 2 covers Basic Atomic and Nuclear Phys...
4,"Why is the text ""This page intentionally left ...","The text ""This page intentionally left blank"" ..."


In [13]:
# Display the DataFrame
qa_df.head()

,Question,Answer
0,What is the process for a first-time user to a...,"For a first-time user, the process to access t..."
1,What are the contact details provided for tech...,The contact details provided for technical ass...
2,What is the role of physics in nuclear medicine?,"According to the table of contents, the role o..."
3,What topics are covered in Chapter 2?,Chapter 2 covers Basic Atomic and Nuclear Phys...
4,"Why is the text ""This page intentionally left ...","The text ""This page intentionally left blank"" ..."


In [14]:
# Display the DataFrame
qa_df.tail()

,Question,Answer
95,How many pages are intentionally left blank ac...,"The text mentions ""This page intentionally lef..."
96,What is a radiopharmaceutical or radiotracer?,"A radiopharmaceutical, or more commonly called..."
97,What is the basic process involved in a nuclea...,"In a nuclear medicine study, a radiopharmaceut..."
98,What are the two broad classes of nuclear medi...,The two broad classes of nuclear medicine imag...
99,What is the purpose of radionuclide imaging?,The purpose of radionuclide imaging is to obta...


In [15]:
!pip install huggingface_hub datasets